In [131]:
import numpy as np
import math
import random

In [132]:
#Initial Pheromone 
def FirstPheromone(hobs):
    return [[1 for i in range(hobs)] for j in range(hobs)]

# A random graph
def Graph(dim):
    
    b = [(i, str(i)) for i in range(dim)]   
    matrix = []
    
    for i,j in b:
        j=[]
        if i == 0:
            j.append(0)
            matrix.append(j)
        else:
            for k in range(i+1):
                if k == i:
                    j.append(0)
                else:
                    j.append(random.choice([0,1]))
            matrix.append(j)
            
    M = matrix        
    for i in range(len(M)):
        
        for j in range(1,len(M)-len(M[i])+1):
            M[i].append(M[i+j][i]) 
            
    return M

# Eliminate Loops 
def Deloop(x):
    l = len(x)

    for i in range(l):
        for j in range(i+1,l):
            j = l+i-j
            
            if j < len(x):          # list changes as loop is removed 
                if x[i] == x[j]:
                    # print("Delooping:  ", x)
                    [x.remove(k) for k in x[i+1:j+1]]
                    # print("Delooped to: ", x)
    return x

In [133]:
# probability and selection using roulette wheel 

# Calculate cumulative sum for roulette wheel 
def PPi(pi):
    n = len(pi)
    ans = []
    
    for i in range(n):
        ans.append( sum([i for i in pi[:i+1]]) )
    return ans 

# Roulette Wheel 
def Roulette(ppi, Pop):
    n = len(ppi)
    ra = random.random()
    
    for i in range(n):
        if i == 0 :
            if ra > 0 and ra < ppi[i]:
                return Pop[i]
        else:
            if ra > ppi[i-1] and ra < ppi[i]:
                return Pop[i]

In [134]:
# Transition Probability
def TransitionP(connections, alpha, beta):
    """ Takes a list of tuples [(pheromone, distance, hob), ... , (pheromone, distance, hob)]
        return trasition probability for each hob [(probability, hob), ... , (probability, hob)]"""
    
    total = sum( [ (i[0]**alpha)*(i[1]**beta) for i in connections])    # sum heuristic and pheromone info
    prob = [( (((i**alpha)*(j**beta))/total) , k) for i,j,k in connections ]  # probability of each hob
    
    return prob

# pheromone, distance and hob information
def DPH(phermatrix, graph, connections, preshob):
    """ DPH(Distance Pheromone hob)
        takes a pheromone matrix
        graph matrix 
        connections in graph to use 
        present hob in graph
        and return the pheromone, distance and hob [(pheromone, distance, hob), ... , (pheromone, distance, hob)]"""
    
    dph = []
    for i in range(len(connections)):
        if connections[i] > 0:
            dph.append( (phermatrix[preshob][i], 1/graph[preshob][i], i) )
            
    #dph = [(phermatrix[preshob][i], graph[preshob][i], i) for i in range(len(connections)) if connections[i] > 0 ]
    
    return dph

In [135]:
def Nexthob1(phermatrix, graph, preshob, prevhob = [] , alpha = 2, beta = 2):
    
    Cons = graph[preshob].copy()  # obtain all connections for present hob
    if prevhob != -1:
        Cons[prevhob] = 0     # set previous connection to zero equivqlent to removal
    
    #Get hob with corresponding pheromone and distance
    HP = DPH(phermatrix, graph, Cons, preshob)
    
    if len(HP) == 0:    # if a dead end is met 
        # print("HEre")
        if prevhob == -1:
            print("No Transition Possible :::::::")
        node = prevhob
    else:
        # Compute Trasition probability for each hob
        prob = TransitionP(HP, alpha, beta)
        node = Roulette(PPi([i[0] for i in prob]), [i[1] for i in prob]) # select hob with probability

    return node, preshob

def FindPath1(phermatrix, graph, takeoff, destination, prevhob = -1, alpha = 2, beta = 2):
    
    path = []
    path.append(takeoff)
    
    while takeoff != destination:
        
        takeoff, prevhob = Nexthob1(phermatrix, graph, takeoff, prevhob, alpha, beta)
        path.append(takeoff)
        
    Deloop(path)
        
    return path

In [136]:
# Update Pheromone in visited arcs

def UpPheromone(route, matrix):
    """route : route taken
    matrix: pheromone matrix"""
    
    tau = 1/len(route)
    #tau = 1
    
    for i in range(len(route)-1):
        
        matrix[route[i]][route[i+1]] += tau
        matrix[route[i+1]][route[i]] += tau

    return matrix

def UpPheromoneNPL(route, matrix):
    
    for i in range(len(route)-1):
        
        matrix[route[i]][route[i+1]] += 1
        matrix[route[i+1]][route[i]] += 1

    return matrix

# Evaporate pheromone 

def EvPheromone(matrix, rho):
    
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            if matrix[i][j] > 0:
                matrix[i][j] = (1-rho)*matrix[i][j]
            else:
                matrix[i][j] = math.exp(-50)
    return matrix

In [137]:
# Simple Ant Colony Optimiza

def ACO101(Ngraph, ants, starthob, destination, alpha = 1, beta = 3, rho = 0.5, runs = 50, Pher = None):
    
    if Pher == None:
        Pher = FirstPheromone(len(Ngraph))

    prevhob = -1
    
    i = 0
    while i < runs:
        Ants = []
        #print(Gr)
        
        for j in range(ants):
            #print(j)
            stof = FindPath1(Pher, Ngraph, starthob, destination, prevhob, alpha, beta)
            Ants.append(stof)
   
        #Pheromone update
        for j in Ants:
            UpPheromone(j, Pher)
            
        #Pheromone update    
        EvPheromone(Pher, rho)
        
        i += 1 
        
    return np.array(Ants), np.array(Pher)

In [172]:
# Network graph from CI book
SACOCI = [[0,0,0,1,0,1,0],
          [0,0,1,1,0,0,0],
          [0,1,0,1,1,0,0],
          [1,1,1,0,1,1,0],
          [0,0,1,1,0,0,1],
          [1,0,0,1,0,0,1],
          [0,0,0,0,1,1,0]]

ants = 20
alpha = 1
beta = 3
rho = 0.5
S, D = 0, 4
runs = 100

ACO101(SACOCI, ants, S, D, alpha, beta, rho, runs)

(array([[0, 3, 4],
        [0, 3, 4],
        [0, 3, 4],
        [0, 3, 4],
        [0, 3, 4],
        [0, 3, 4],
        [0, 3, 4],
        [0, 3, 4],
        [0, 3, 4],
        [0, 3, 4],
        [0, 3, 4],
        [0, 3, 4],
        [0, 3, 4],
        [0, 3, 4],
        [0, 3, 4],
        [0, 3, 4],
        [0, 3, 4],
        [0, 3, 4],
        [0, 3, 4],
        [0, 3, 4]]),
 array([[7.88860905e-31, 7.88860905e-31, 7.88860905e-31, 6.66666667e+00,
         7.88860905e-31, 2.51187117e-26, 7.88860905e-31],
        [7.88860905e-31, 7.88860905e-31, 1.53039016e-29, 1.53039016e-29,
         7.88860905e-31, 7.88860905e-31, 7.88860905e-31],
        [7.88860905e-31, 1.53039016e-29, 7.88860905e-31, 5.12238940e-26,
         5.12384091e-26, 7.88860905e-31, 7.88860905e-31],
        [6.66666667e+00, 1.53039016e-29, 5.12238940e-26, 7.88860905e-31,
         6.66666667e+00, 6.75355654e-27, 7.88860905e-31],
        [7.88860905e-31, 7.88860905e-31, 5.12384091e-26, 6.66666667e+00,
         7.88860905e-

In [171]:
# Extended Double bridge matrix
# solution = [ 0,  9, 10, 12, 13,  8] or [ 0,  9, 14, 17, 13,  8]

EDBM = [[0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0],
        [1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
        [0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
        [0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
        [0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0],
        [0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0],
        [0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0],
        [0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0],
        [0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0],
        [1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0],
        [0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,1,0,0,0],
        [0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0],
        [0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,1,0,0],
        [0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0],
        [0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,1],
        [0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0],
        [0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0],
        [0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,1],
        [0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0]]

ants = 20
alpha = 1
beta = 3
rho = 0.5
S, D = 0, 8
runs = 10

ACO101(EDBM, ants, S, D, alpha, beta, rho, runs)

(array([list([0, 9, 10, 12, 13, 8]), list([0, 1, 2, 3, 4, 5, 6, 7, 8]),
        list([0, 1, 2, 3, 4, 5, 6, 7, 8]),
        list([0, 1, 2, 3, 4, 5, 6, 7, 8]),
        list([0, 1, 2, 3, 4, 5, 6, 7, 8]),
        list([0, 1, 2, 3, 4, 5, 6, 7, 8]),
        list([0, 1, 2, 3, 4, 5, 6, 7, 8]), list([0, 9, 14, 18, 17, 13, 8]),
        list([0, 1, 2, 3, 4, 5, 6, 7, 8]),
        list([0, 1, 2, 3, 4, 5, 6, 7, 8]),
        list([0, 1, 2, 3, 4, 5, 6, 7, 8]),
        list([0, 1, 2, 3, 4, 5, 6, 7, 8]),
        list([0, 1, 2, 3, 4, 5, 6, 7, 8]),
        list([0, 1, 2, 3, 4, 5, 6, 7, 8]),
        list([0, 1, 2, 3, 4, 5, 6, 7, 8]),
        list([0, 1, 2, 3, 4, 5, 6, 7, 8]),
        list([0, 1, 2, 3, 4, 5, 6, 7, 8]), list([0, 9, 10, 11, 12, 13, 8]),
        list([0, 1, 2, 3, 4, 5, 6, 7, 8]),
        list([0, 1, 2, 3, 4, 5, 6, 7, 8])], dtype=object),
 array([[9.76562500e-04, 1.97569444e+00, 9.76562500e-04, 9.76562500e-04,
         9.76562500e-04, 9.76562500e-04, 9.76562500e-04, 9.76562500e-04,
         9.7